# Домашнее задание №8

# https://www.kaggle.com/c/mercari-price-suggestion-challenge 
# https://www.kaggle.com/c/home-credit-default-risk

Выбрать одно из двух соревнований. Выбирайте по данным, с которыми вам интереснее работать.

In [1]:
from __future__ import print_function


import time
import re
from collections import defaultdict

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, LabelEncoder, MinMaxScaler,  Imputer, LabelBinarizer, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Ансамбли

import xgboost as xgb
import lightgbm as lgb

%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 8)
pd.options.display.float_format = '{:.2f}'.format

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
df = pd.read_csv('input/train.tsv', sep='\t')

In [ ]:
df.info()

### Используйте параметр nrows, чтобы уменьшить выборку и сделать базовый разведочный анализ данных

In [ ]:
# df = pd.read_csv('data/train.tsv', nrows=999999, sep='\t')
# mercari_prices = pd.read_csv('data/train.tsv', nrows=249, sep='\t')
mercari_prices = pd.read_csv('input/train.tsv', nrows=9999, sep='\t')

In [ ]:
# def prepare_data(N, subcats=True):
#     df = pd.read_csv('input/train.tsv', nrows=N, sep='\t')
#     if subcats:
#         cats = df.category_name.str.split('/', expand=True)
#         df['cat1'] = cats[0]
#         df['cat2'] = cats[1]
#         df['cat3'] = cats[2]
#         df = df.drop(['category_name'], axis=1)
# #     df.head()

#     df['brand_id'] = pd.Categorical(df['brand_name']).codes

#     return df

def load_data(N):
    df = pd.read_csv('input/train.tsv', nrows=N, sep='\t')
#     if subcats:
#         cats = df.category_name.str.split('/', expand=True)
#         df['cat1'] = cats[0]
#         df['cat2'] = cats[1]
#         df['cat3'] = cats[2]
#         df = df.drop(['category_name'], axis=1)
# #     df.head()

#     df['brand_id'] = pd.Categorical(df['brand_name']).codes

    return df



# df = mercari_prices.copy()
# df.head()

# # # df.category_name.str.split('/')
# # for category in df.category_name.str.split('/').apply(pd.Series).stack().reset_index(drop=True):
# #     df[category] = df.category_name.str.contains(category)
# # df = df.drop(['category_name'], axis=1)

# # df.category_name.str.split('/', expand=True)

# # # df.head().T
# # # df.category_name.str.split('/', expand=True)
# # # df.
# # # df['categories'] = pd.Series(df.category_name.str.split('/', expand=True))
# cats = df.category_name.str.split('/', expand=True)
# df['cat1'] = cats[0]
# df['cat2'] = cats[1]
# df['cat3'] = cats[2]
# df = df.drop(['category_name'], axis=1)
# df.head()

# df = prepare_data(17000)
# df = prepare_data(9999)
df = load_data(9999)

df[df.item_description.str.contains('\d', regex=True)][:10]
df[df.name.str.contains('\d', regex=True)][:10]

In [ ]:
# df.groupby('brand_name').min()['price'][:10].plot.bar()
# df.groupby('brand_name').mean()['price'][:10].plot.bar()
# df.groupby('brand_name').max()['price'][:10].plot.bar()

df['count'] = 1
brands = list(df.groupby('brand_name').count()['count'].nlargest(10).index)

fig, ax = plt.subplots(1, 1, figsize=(12,4))

# import seaborn as sns

# # # sns.violinplot(x='part', y='rating', order=parts, data=df)
# sns.violinplot(x='brand_name', y='price', data=df[df['brand_name'].isin(brands)], ax=ax)
# sns.despine(left=True)

df.loc[:, ['brand_name']]

In [ ]:
df.head()

In [ ]:
!head input/sample_submission.csv

In [ ]:
df_test = pd.read_csv('input/test.tsv', sep='\t')

In [ ]:
df_test.info()

## 1. Сделать baseline submission

  ### 1.1 Исследование признак price.

In [ ]:
df.price.describe(percentiles=[.05, .25, .5, .75, .95])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,4))

df1 = load_data(9999)
s1 = df1['price'] # .hist()
s1.hist(ax=ax[0])  # (bins=100)

s2 = df1[df1['price'] < 94.5]['price'] # .hist()
s2.hist(ax=ax[1])  # (bins=100)

### 1.2 Исследование признака price в зависимости от brand_name или других признаков

In [ ]:
# corr_matrix = df.corr()
# display(corr_matrix)
# df.drop(['brand_name', 'price'], axis=1).corr(method='pearson').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)
df['brand_id'] = pd.Categorical(df['brand_name']).codes
df.dropna().corr(method='pearson').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

In [ ]:
sns.pairplot(df, kind="reg")

## 2. Реализовать цикл анализа

### 2.1 признаки -> модель -> настройка параметров -> лучшая модель и ее значение метрики качества на кросс-валидации

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
models = {
    'log_reg': LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000),
    'svc': SVC(kernel='linear', C=100),
    'tree': DecisionTreeClassifier(),
    'knei_reg': KNeighborsRegressor(),
    'xgb': xgb.XGBClassifier(),
}



class Ensemble(BaseEstimator):  
    """An example of classifier"""

    def __init__(self):
        self.models = list(models.values())
    
#     def __init__(self, models):
# #         print('INIT', name)
#         self.models = models
    
    def fit(self, X, y=None):
        for model in self.models:
            model.fit(X, y)
        return self

    def predict(self, X, y=None):
        ___ = [
            int(np.mean(items))
            for items in 
            zip(*[model.predict(X) for model in self.models])
        ]
#         print('___', ___)
        return ___
       
    def score(self, X, y=None):
        return(sum(self.predict(X)))




In [ ]:
from sklearn.preprocessing import FunctionTransformer

from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error

def prepare_data(N, subcats=True):
    df = pd.read_csv('input/train.tsv', nrows=N, sep='\t')
    return df

_models = models.copy()
_models['ensemble'] = Ensemble()

class Router(BaseEstimator):  
#     """An example of classifier"""

    def __init__(self, name=None, estimator=None, params={}, x_pred=None, y_true=None, **kwargs):
#         print('INIT', name)
        self.name = name or 'svc' # type(models[model])
        self.estimator = estimator
        self.x_pred = x_pred
        self.y_true = y_true
        self.params = params  # or {}
#         self.model_instance = models[self.name]
        self.initial_params = _models[self.name].get_params()
#         print('PARAMS 1', name, self.name, self.model_instance, self.initial_params)
#         params.update()
        self.model_instance = type(_models[self.name])(**self.params)
    
#     def get_model_instance(self):
#         return type(self.model_instance)(**self.params)
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
#         print('SET NAME', value)
        self._name = value
        params = _models[self.name].get_params()  # getattr(self, 'initial_params', {}).copy() 
        params.update(getattr(self, 'params', {}))
#         print('PARAMS 2', value, getattr(self, 'name', None), getattr(self, 'model_instance', None), params)
        self.model_instance = type(_models[self._name])(**params)
#         self.model_instance = type(_models[self._name])(**getattr(self, 'params', {}))
    
    @property
    def params(self):
        return self._params
    
    @params.setter
    def params(self, value):
#         print('SET PARAMS', value)
        self._params = value
        params = getattr(self, 'initial_params', {}).copy()
        params.update(getattr(self, 'params', {}))
#         print('PARAMS 3', value, getattr(self, 'name', None), getattr(self, 'model_instance', None), params)
        self.model_instance = type(_models[self._name])(**params)
#         self.model_instance = type(_models[self._name])(**getattr(self, 'params', {}))
# #         print('SET PARAMS', value)

    
    def fit(self, X, y=None):
#         self.get_model_instance().fit(X, y)
#         return self
#         self.model_instance = models[self.name]
        self.model_instance.fit(X, y)
        return self

    def predict(self, X, y=None):
#         return self.get_model_instance().predict(X)
#         self.model_instance = models[self.name]
        return list(map(int, self.model_instance.predict(X)))
       
    def score(self, X, y=None):
#         print('SCORE', self.model_instance.predict(X).shape, y.shape)
        error = mean_squared_error(self.model_instance.predict(X), y)
        score = 1 / (1 + error)
#         print(self.name, score, error)
#         print('SCORE', self.name, score, self.model_instance.score(X, y))
        print('.', end='')
        return score
    
#         error = sum([1 if 0.1 < abs(y1 - y2) else 0 for y1, y2 in zip(self.model_instance.predict(X), y)])/y.shape[0]
#         score = 1 / (1 + error)
#         print(self.name, score, error)
#         return score

def category_splitter(df):
    df = df[:]
    cats = df.category_name.str.split('/', expand=True)
#     top, middle, bottom = df.category_name.str.split('/', expand=True)
#     df = df.drop(['category_name'], axis=1)
    
#     df = pd.DataFrame(dict(**df.to_dict(), cat1=cats[0], cat2=cats[1], cat3=cats[2]))
    df = pd.DataFrame(dict(**df.to_dict(), top=cats[0], middle=cats[1], bottom=cats[2]))
#     df = pd.DataFrame(dict(**df.to_dict(), top=top, middle=middle, bottom=bottom))
    
    return df
# category_splitter = completor
completor = category_splitter

def _execute_analyze_cycle(model, dataframe, *, x=None, y=None, params=None, features=None, preparers=None, matched=False):
    preparers = preparers or []
    ret = dict()
    
    def feature_selector(x, features=None):
        if features:
            x = x.loc[:, features.split(',')]
        return x
    
    dataframe = dataframe.dropna()
    
    x = x or dataframe.drop(['price'], axis=1)
    y = y or np.ravel(dataframe.loc[:, ['price']])


    preparation = Pipeline(steps = [
        ('feature_selector', FunctionTransformer(feature_selector, validate=False)),

        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
        ('scaler', StandardScaler()),            
    ])
    
    x_train, x_test, y_train, y_test = train_test_split(
        x,
        y,
        test_size=0.20, 
        random_state=42
    )

    y_true = np.ravel(y_test)
#     y_pred = search_cv.predict(x_test)


    _x_train, _x_test, _y_train, _y_test = train_test_split(
        preparation.fit_transform(x),
        y,
        test_size=0.20, 
        random_state=42
    )
    _y_true = np.ravel(_y_test)


    def estimator(model):
        print(model)
#         x = preparation.fit_transform(x_test)
        model.fit(_x_train, _y_train)
#         print(model.predict(x).shape, y_true.shape)
        return accuracy_score(y_pred=list(map(int, model.predict(_x_test))), y_true=_y_true)

#     pipeline = Pipeline(steps = [
#         ('completor', FunctionTransformer(completor, validate=False)),
# #             ('x_selector', FunctionTransformer(sample_selector, validate=False, )),
# #             ('y_selector', FunctionTransformer(sample_selector, pass_y=True, validate=False, )),
#         ('feature_selector', FunctionTransformer(feature_selector, validate=False)),

#         ('imputer', SimpleImputer(strategy='most_frequent')),
#         ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
#         ('scaler', StandardScaler()),            
# #         ('model', models[model]) if isinstance(model, str) else model,
#         ('model', Router('log_reg', estimator=None, x_pred=x_test, y_true=y_true))
#     ])

    steps = [
# #         ('completor', FunctionTransformer(completor, validate=False)),
# #             ('x_selector', FunctionTransformer(sample_selector, validate=False, )),
# #             ('y_selector', FunctionTransformer(sample_selector, pass_y=True, validate=False, )),
#         ('feature_selector', FunctionTransformer(feature_selector, validate=False)),
    ] + preparers + [
#             ('x_selector', FunctionTransformer(sample_selector, validate=False, )),
#             ('y_selector', FunctionTransformer(sample_selector, pass_y=True, validate=False, )),
        ('feature_selector', FunctionTransformer(feature_selector, validate=False)),
        
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
        ('scaler', StandardScaler()),            
#         ('model', models[model]) if isinstance(model, str) else model,
        ('model', Router('log_reg', estimator=None, x_pred=x_test, y_true=y_true))
    ]
#     print('PS', preparers)
    pipeline = Pipeline(steps=steps)

    grid = params or {}

#     search_cv = RandomizedSearchCV(model, param_grid, cv=5, scoring='accuracy')

    search_cv = GridSearchCV(pipeline, grid, cv=3, iid=True)             
    search_cv.fit(x_train, np.array(y_train))

    ret['search_cv'] = search_cv

    y_pred = search_cv.predict(x_test)
    
#     print('PRED', y_pred)
#     print('TRUE', y_true)

    ret['accuracy_score'] = 0  # accuracy_score(y_pred=y_pred, y_true=y_true)

    return ret


grid0 = {
# #     'model__penalty': ['l2', 'none'],
#     'model__max_iter': [250], # [100, 250, 500, 1000],
    
#     'x_selector__kw_args': [{'count': count} for count in [250, 500, 750]],
#     'y_selector__kw_args': [{'count': count} for count in [250, 500, 750]],
    'feature_selector__kw_args': [
#         {'features': 'brand_name'}, 
#         {'features': 'cat1'}, 
#         {'features': 'cat2'}, 
#         {'features': 'cat3'},
#         {'features': 'cat1,cat2,cat3'},
        {'features': 'category_name'},
#         {'features': 'shipping'},
    ],
    'model__name': list(_models.keys()),
#     'model__name': ['knei_reg'], # list(models.keys()),  # ['log_reg', 'svc', 'tree'], # [100, 250, 500, 1000],
# #     'model__params': [{'C': C} for C in [1, 10, 100, 1000, 10000]],
#     'model__params': [{'n_neighbors': param} for param in range(2, 6)],
# #     'model__params': [{'max_iter': param} for param in range(100, 1000, 100)],
}



grid = {
# #     'model__penalty': ['l2', 'none'],
#     'model__max_iter': [250], # [100, 250, 500, 1000],
    
#     'x_selector__kw_args': [{'count': count} for count in [250, 500, 750]],
#     'y_selector__kw_args': [{'count': count} for count in [250, 500, 750]],
    'feature_selector__kw_args': [
#         {'features': 'brand_name'}, 
        
#         {'features': 'cat1'}, 
#         {'features': 'cat2'}, 
#         {'features': 'cat3'},
#         {'features': 'cat1,cat2,cat3'},
#         {'features': 'category_name'},
        
#         {'features': 'top'}, 
#         {'features': 'middle'}, 
#         {'features': 'bottom'},
#         {'features': 'top,middle,bottom'},
#         {'features': 'category_name'},

#         {'features': 'shipping'},
        
        {'features': 'middle'},
    ],
    'model__name': list(_models.keys()),
#     'model__name': ['knei_reg'], # list(models.keys()),  # ['log_reg', 'svc', 'tree'], # [100, 250, 500, 1000],
# #     'model__params': [{'C': C} for C in [1, 10, 100, 1000, 10000]],
#     'model__params': [{'n_neighbors': param} for param in range(2, 6)],
# #     'model__params': [{'max_iter': param} for param in range(100, 1000, 100)],
}

# SVC(kernel='linear', C=100)

preparers = [
    ('category_splitter', FunctionTransformer(category_splitter, validate=False))
]

# fig, ax = plt.subplots(1, 2, figsize=(12,4))

# def extract_target_into_x(data, target):
# #     pd.DataFrame(data['search_cv'].cv_results_)

# #     data = data['search_cv'].cv_results_

#     data

# #     params = list(dict(data)['params'])[0]
# #     param_name = 'param_' + list(params.keys())[0]
# #     param_name

#     # # data = {k:[(list(vv.values())[0] if k == param_name else vv) for vv in v] for k, v in data.items()}
#     # data = {k:[(list(','.join(vv.values())) if k == param_name else vv) for vv in v] for k, v in data.items()}
#     data = {k:[(','.join(list(vv.values())) if k == param_name else vv) for vv in v] for k, v in data.items()}

#     data = pd.DataFrame(data)
#     data

# #     params.pop('model__name')

#     data['x'] = data[target]

    # data.groupby(['param_model__name', param_name]).sum()['mean_test_score'].unstack().plot.bar()    

# # print('BEGIN', end='')
# # ret0 = _execute_analyze_cycle('log_reg', prepare_data(300), params=grid0)
# # print('END')
# print('BEGIN', end='')
# ret = _execute_analyze_cycle('log_reg', prepare_data(300), params=grid, preparers=preparers)
# print('END')
# # data = ret

fig, ax = plt.subplots(1, 2, figsize=(12,4))

def extract_target_from_search_cv_into_x(data, target):
    data = data['search_cv'].cv_results_
    data = {k:[(','.join(list(vv.values())) if k == target else vv) for vv in v] for k, v in data.items()}
    data = pd.DataFrame(data)
    data['x'] = data[target]
    
    return data

     
# # (extract_target_from_search_cv_into_x(ret0, 'param_feature_selector__kw_args')
# #  .groupby(['param_model__name', 'x']).sum()['mean_test_score'].unstack().plot.bar(ax=ax[0]))
# (extract_target_from_search_cv_into_x(ret, 'param_feature_selector__kw_args')
#  .groupby(['param_model__name', 'x']).sum()['mean_test_score'].unstack()
# #  .loc[:, ['category_name', 'cat1,cat2,cat3', 'cat1', 'cat2', 'cat3']]
#  .loc[:, ['category_name', 'top,middle,bottom', 'top', 'middle', 'bottom']]
#  .plot.bar(ax=ax))
     
# plt.legend(loc='best')




# pd.DataFrame(data['search_cv'].cv_results_)

# data = data['search_cv'].cv_results_

# data

# params = list(dict(data)['params'])[0]
# param_name = 'param_' + list(params.keys())[0]
# param_name

# # # data = {k:[(list(vv.values())[0] if k == param_name else vv) for vv in v] for k, v in data.items()}
# # data = {k:[(list(','.join(vv.values())) if k == param_name else vv) for vv in v] for k, v in data.items()}
# data = {k:[(','.join(list(vv.values())) if k == param_name else vv) for vv in v] for k, v in data.items()}

# data = pd.DataFrame(data)
# data

# params.pop('model__name')

# data.groupby(['param_model__name', param_name]).sum()['mean_test_score'].unstack().plot.bar()
# plt.legend(loc='best')

def __execute_analyze_cycle(model=None, features=None, preparers=None, preparer=None):
    preparers = [] if preparer is None else [
        (preparer.__name__, FunctionTransformer(preparer, validate=False))
#         ('category_splitter', FunctionTransformer(category_splitter, validate=False))
    ]
    x='param_feature_selector__kw_args'
    df = prepare_data(300)
    models = model if model is not None else list(_models.keys())
    features = features if features is not None else df.columns
    preparers = preparers or []
    grid = {    
        'feature_selector__kw_args': [{'features': feature} for feature in features],
        'model__name': models,
    }
    ret = _execute_analyze_cycle('log_reg', df, params=grid, preparers=preparers)
    ret = extract_target_from_search_cv_into_x(ret, x)
    ret['features'] = ret['param_feature_selector__kw_args']
    ret['models'] = ret['param_model__name']
    ret['score'] = ret['mean_test_score']
#     ret = ret.loc[:, features]
    return ret

def ___execute_analyze_cycle(models=None, N=None, **kwargs):
#     all_preparers = [
#         (k, FunctionTransformer(v, validate=False)) for k, v in kwargs.items() 
#     ]
    preparers = []
    bodies = []
#     print('KW', type(kwargs), list(kwargs.items()))
    for name, body in kwargs.items():
#         print('???', name, body)
        if body not in bodies:
            bodies.append(body)
            preparers.append((name, FunctionTransformer(body, validate=False)))
#         else:
#             print('!', bodies, body)
#     preparers = [] if preparer is None else [
#         (preparer.__name__, FunctionTransformer(preparer, validate=False))
# #         ('category_splitter', FunctionTransformer(category_splitter, validate=False))
#     ]
#     print('PS0', preparers)
    x='param_feature_selector__kw_args'
    df = prepare_data(N or 300)
    models = models if models is not None else list(_models.keys())
    features = list(kwargs.keys()) # features if features is not None else df.columns
#     preparers = preparers or []
    grid = {    
        'feature_selector__kw_args': [{'features': feature} for feature in features],
        'model__name': models,
    }
    ret = _execute_analyze_cycle('log_reg', df, params=grid, preparers=preparers)
    ret = extract_target_from_search_cv_into_x(ret, x)
#     ret['features'] = ret['param_feature_selector__kw_args']
#     ret['models'] = ret['param_model__name']
#     ret['score'] = ret['mean_test_score']
    ret = ret.rename(columns=dict(
        param_feature_selector__kw_args='features',
        param_model__name='models',
        mean_test_score='score',
    ))
    print(ret.columns)
#     ret = ret.loc[:, features]
    ret = ret.groupby(['models', 'features']).sum()['score'].unstack().loc[:, features]
    return ret


     
# features = ['category_name', 'bottom']
# (__execute_analyze_cycle(features=features, preparer=category_splitter)
# #  .groupby(['param_model__name', 'features']).sum()['mean_test_score'].unstack()
#   .groupby(['models', 'features']).sum()['mean_test_score'].unstack() 
# #  .loc[:, ['category_name', 'cat1,cat2,cat3', 'cat1', 'cat2', 'cat3']]
# #  .loc[:, ['category_name', 'top,middle,bottom', 'top', 'middle', 'bottom']]
#  .loc[:, features]
#  .plot.bar(ax=ax[0]))

     
def category_splitter(df):
#     if 'top' in df.columns:
#         return df
    
#     cats = df.category_name.str.split('/', expand=True)
    top, middle, bottom = df.category_name.str.split('/', expand=True).to_dict().values()
#     print(cats)
#     print(t,m,b)
#     return pd.DataFrame(dict(**df.to_dict(), top=cats[0], middle=cats[1], bottom=cats[2]))
    return pd.DataFrame(dict(**df.to_dict(), top=top, middle=middle, bottom=bottom))
    
def first(df):
    return pd.DataFrame(dict(**df.to_dict(), first=df.category_name.str[0]))

cs = category_splitter
___execute_analyze_cycle(N=200, top=cs, middle=cs, bottom=cs, first=first).plot.bar(ax=ax[1])
    
# def first(df):
#     return pd.DataFrame(dict(**df.to_dict(), first=df.category_name.str[0]))

# features = ['first',]
# (__execute_analyze_cycle(features=features, preparer=first)
#  .groupby(['models', 'features']).sum()['score'].unstack() 
#  .loc[:, features]
#  .plot.bar(ax=ax[1]))



In [ ]:
df = prepare_data(50000)
df['number'] = df.name.str.extract(r"(\d+)")

df[df['number'].notna()].loc[:, ['number', 'name', 'price']]

In [ ]:
# def extract_target_from_search_cv_into_x(data, target):
# #     pd.DataFrame(data['search_cv'].cv_results_)

#     data = data['search_cv'].cv_results_

#     data

# #     params = list(dict(data)['params'])[0]
# #     param_name = 'param_' + list(params.keys())[0]
# #     param_name

#     # # data = {k:[(list(vv.values())[0] if k == param_name else vv) for vv in v] for k, v in data.items()}
#     # data = {k:[(list(','.join(vv.values())) if k == param_name else vv) for vv in v] for k, v in data.items()}
#     data = {k:[(','.join(list(vv.values())) if k == target else vv) for vv in v] for k, v in data.items()}

#     data = pd.DataFrame(data)

# #     params.pop('model__name')

#     data['x'] = data[target]
    
#     return data


# (extract_target_from_search_cv_into_x(ret, 'param_feature_selector__kw_args')
#  .groupby(['param_model__name', 'x']) .sum()['mean_test_score'].unstack().plot.bar())


# # data = ret

# # pd.DataFrame(data['search_cv'].cv_results_)

# # data = data['search_cv'].cv_results_

# # # data

# # params = list(dict(data)['params'])[0]
# # data
# # params
# # # param_name = 'param_' + list(params.keys())[0]
# # # param_name

# # def merge(key, values):
# #     if key == param_name:
# #         print('VALUES', values)
# #         return ','.join(values)
# #     else:
# #         return values

# # # # data = {k:[(list(vv.values())[0] if k == param_name else vv) for vv in v] for k, v in data.items()}
# # # data = {k:[merge(k, (list(vv.values())) if k == param_name else vv) for vv in v] for k, v in data.items()}
# # data = {k:[(','.join(list(vv.values())) if k == param_name else vv) for vv in v] for k, v in data.items()}

# # data
# # data = pd.DataFrame(data)
# # # data

# # # params.pop('model__name')


# # data.groupby(['param_model__name', param_name]) .sum()['mean_test_score'].unstack().plot.bar()


In [ ]:
# def prepare_data(N, subcats=True):
#     df = pd.read_csv('input/train.tsv', nrows=N, sep='\t')
# #     if subcats:
# #         cats = df.category_name.str.split('/', expand=True)
# #         df['cat1'] = cats[0]
# #         df['cat2'] = cats[1]
# #         df['cat3'] = cats[2]
# #         df = df.drop(['category_name'], axis=1)
# # #     df.head()

# #     df['brand_id'] = pd.Categorical(df['brand_name']).codes

#     return df




# from sklearn.preprocessing import FunctionTransformer
# # na = df[features].isna().apply(lambda s: s.value_counts(), axis=0)
# # na
# # list(filter(lambda t: t[0] > 0, map(lambda col: (na.loc[True, col], col), na.columns)))


# # df = mercari_prices.dropna()


# # # X_train, y_train = df.drop(['price'], axis=1), df.loc[:, ['price']]

# # from sklearn.model_selection import train_test_split

# # preparation = Pipeline(steps = [
# #     ('imputer', SimpleImputer(strategy='most_frequent')),
# #     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
# #     ('scaler',StandardScaler()),
# # ])

# # x_train, x_test, y_train, y_test = train_test_split(
# #     preparation.fit_transform(df.drop(['price'], axis=1)), 
# #     np.ravel(df.loc[:, ['price']]),
# #     test_size=0.20, 
# #     random_state=42
# # )


# # # x_train, x_test, y_train, y_test = train_test_split(
# # #     df.drop(['price'], axis=1), 
# # #     np.ravel(df.loc[:, ['price']]),
# # #     test_size=0.20, 
# # #     random_state=42
# # # )
    
# # display('SHAPE', mercari_prices.shape, df.shape)
# # x_train, y_train
# pass

# def execute_analyze_cycle(models, dataframe, *, x=None, y=None, params=None, features=None, matched=False):
#     ret = dict()
    
# #     def sample_selector(x, count=None, *args, **kwargs):
# # #         print('SELECTOR', x, args, kwargs)
# # #         print('SELECTOR', x, features)
# #         print('SAMPLE', x[:10], count, args, kwargs)
# # #         if count:
# # #             x = x.iloc[:count]
# # #         if count:
# # #             y = y[:count]
# #         return x
        
# #     def selector(X, *args, **kwargs):
#     def feature_selector(x, features=None):
# #         print('SELECTOR', x, args, kwargs)
# #         print('SELECTOR', x, features)
#         print('FEATURE', type(x), features)
#         if features:
#             x = x.loc[:, features]
#         return x

    
#     selection = Pipeline(steps = [
#         ('feature_selector', FunctionTransformer(feature_selector, validate=False)),
#     ])
        
#     preparation = Pipeline(steps = [
# #         ('selector', FunctionTransformer(selector, validate=False)),
# #         ('selector', FunctionTransformer(None)),
#         ('imputer', SimpleImputer(strategy='most_frequent')),
#         ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
# #         ('imputer', SimpleImputer(strategy='most_frequent')),
#         ('scaler', StandardScaler()),
#     ])
    

#     if type(models) is list: 
#         models = {str(i): model for i, model in enumerate(models)}
#     elif type(models) is dict:
#         pass
#     else:
#         models = {'0': models}
        
#     predictions = []
#     accuracy_scores = []
    
#     y_true = []
        
#     for name, model in models.items():
#         x = x or dataframe.drop(['price'], axis=1)
#         if type(features) is list:
#             x = x.loc[:, features]
#         elif type(features) is dict and name in features:
#             not_found = [feature for feature in features[name] if feature not in x.columns]
#             if not_found:
#                 print('features {} not found in columns {}'.format(not_found, x.columns))
#                 return
#             x = x.loc[:, features[name]]
# #             if name in x.columns:
# #                 x = x.loc[:, features[name]]
# #                 print('FS', features[name])
# #             else:
# #                 print('features {} not found in columns {}'.format(features[name], x.columns))
#         y = y or np.ravel(dataframe.loc[:, ['price']])

#     #     x = x.drop(['train_id'], axis=1)

# #         print('SELECTED', selection.transform(x))
    
#         x_train, x_test, y_train, y_test = train_test_split(
#     #         x.drop(['train_id'], axis=1), 
#     #         preparation.fit_transform(x.drop(['train_id'], axis=1)), 
# #             preparation.fit_transform(x), 
#             x,
# #             preparation.transform(x),
# #             preparation.fit_transform(selection.transform(x)),
#             y,
#             test_size=0.20, 
#             random_state=42
#         )

#     #     ids_train, ids_test = x_train.loc[:, ['train_id']], x_test.loc[:, ['train_id']]

#     #     ret['ids_train'] = ids_train
#     #     ret['ids_test'] = ids_test
#         ret['x_test'] = x_test
#         ret['y_test'] = y_test

#         y_true = np.ravel(y_test)

#     #     x_train, x_test = map(preparation.fit_transform, (x_train, x_test))


        
# #         if type(features) is list:
# #             x = x.loc[:, features]

# #         def selector(*args, **kwargs):
# #             print('SELECTOR', args, kwargs)
# #             return []
        
#         pipeline = Pipeline(steps = [
#         # #     ('imputer', SimpleImputer()),
#         #     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
#         #     ('scaler', StandardScaler()),
#         # #     ('pca', PCA(n_components=2)),
#         # #         ('pca', PCA()),
#         #     ('log_reg', log_reg),
            
# #             ('x_selector', FunctionTransformer(sample_selector, validate=False, )),
# #             ('y_selector', FunctionTransformer(sample_selector, pass_y=True, validate=False, )),
#             ('feature_selector', FunctionTransformer(feature_selector, validate=False)),
            
#         ('imputer', SimpleImputer(strategy='most_frequent')),
#         ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
# #         ('imputer', SimpleImputer(strategy='most_frequent')),
#         ('scaler', StandardScaler()),

            
            
# #             ('selector', FunctionTransformer(selector)), 
#             ('model', model),
#         ])

#         grid = params or {}

#     #     search_cv = RandomizedSearchCV(model, param_grid, cv=5, scoring='accuracy')

#         search_cv = GridSearchCV(pipeline, grid, cv=3, iid=True)     

# #     #     search_cv.fit(np.array(x_train.drop(['train_id'], axis=1)), np.array(y_train))
# #         search_cv.fit(np.array(x_train), np.array(y_train))mv
        
# #         print('Y_TRAIN', y_train[:10])
    
#         search_cv.fit(x_train, np.array(y_train))

#         search_cv.best_estimator_

#         ret['search_cv'] = search_cv
        
#         ret['best_estimator'] = search_cv.best_estimator_
        
#         ret['best_score'] = search_cv.best_score_
#     #     display(search_cv.best_score_)

#     #     ret['accuracy_score'] = accuracy_score(y_pred = search_cv.predict(x_test), y_true = np.ravel(y_test))
#         y_pred = search_cv.predict(x_test)
#         predictions.append(y_pred)
#         accuracy_scores.append(accuracy_score(y_pred=y_pred, y_true=y_true))
        
# #     y_true = np.ravel(y_test)
# #     display(x_test.shape, y_pred.shape, y_true.shape)
# #     display(type(y_pred == y_true), y_pred == y_true)
# #     print(y_pred, y_true)
#     from collections import Counter
#     y_pred = [
# #         list(Counter(items).keys())[0]
# #         Counter(items).most_common()[0][0]
#         int(np.mean(items))
#         for items in 
#         zip(*predictions)]
#     ret['y_pred'] = y_pred
#     ret['y_true'] = y_true
#     ret['accuracy_score'] = accuracy_score(y_pred=y_pred, y_true=y_true)

# #     if matched:
# #         display(df[df['train_id'].isin(list(ids_test[y_pred == y_true]['train_id']))])
    
# #     display(ret['accuracy_score'])
    
#     ret['accuracy_scores'] = accuracy_scores
#     ret['predictions'] = predictions
#     return ret

# def execute_mutly_analyze_cycles(Ns, models, *args, trace=False, **kwargs):  # FIXME
# # def execute_mutly_analyze_cycles(Ns, model, dataframe, *args, **kwargs):
#     scores = pd.DataFrame({'size': [], 'best_score': [], 'accuracy_score': []})
# # display(scores)
# # for i, N in enumerate(range(1000, 10000, 1000)):
# # for N in [3000, 3099, 3299, 3499, 3699, 3999, 4200]:
# # for N in [3780, 3830, 3960, 3390, 3999, 4410]:
# # for N in range(3780, 3810):
#     for i, N in enumerate(Ns):
#         if trace:
#             print(N)

#         dataframe = prepare_data(N, subcats=True)
#         dataframe = dataframe.dropna()
# #                 dataframe = dataframe.dropna()

#         score = execute_analyze_cycle(models, dataframe, *args, **kwargs)
#         if not score:
#             continue

#         scores.loc[i] = (N, score['best_score'], score['accuracy_score'])
    
#     return scores

# # features_set= [
# #     {'svc': ['brand_id']},
# #     {'svc': ['brand_name']},
# #     {'svc': ['name']},
# #     {'svc': ['shipping']},
# # ]
# # models = {
# #     'log_reg': LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)),
# #     'svc': SVC(kernel='linear', C=100),
# #     'tree': DecisionTreeClassifier(),
# # }

# Ns = [1333]
# by_model_and_feature = pd.DataFrame(dict.fromkeys(['feature', 'model', 'accuracy_score'], []))
# # features_set = [{'svc': [column]} for column in df.columns if column not in ['price']]
# features_set = [{model[0]: [column] for model in models} for column in df.columns if column not in ['price']]


# # for features in features_set:    
# #     for model_name, model in models.items():
# #         feature = features[model_name][0]
# #         scores = execute_mutly_analyze_cycles(
# #             Ns,
# #     #                 range(250, 500, 250),
# #             {model_name: model}, 
# #     #         features={'svc': ['brand_id']}
# #             features=features
# #         )
# #         display(features, scores)
# #         if scores.shape[0] > 0:
# #             by_model_and_feature.loc[by_model_and_feature.shape[0]] = (feature, model_name, scores.iloc[0]['accuracy_score'])
# # #         display(features, scores)
# # display(by_model_and_feature)

# # for features in features_set:
# #     scores = execute_mutly_analyze_cycles(
# #         Ns,
# # #                 range(250, 500, 250),
# #         {'svc': SVC(kernel='linear', C=100)}, 
# # #         features={'svc': ['brand_id']}
# #         features=features
# #     )
# #     display(features, scores)


In [ ]:
# from sklearn.preprocessing import FunctionTransformer

# from sklearn.base import BaseEstimator
# from sklearn.metrics import mean_squared_error

# def prepare_data(N, subcats=True):
#     df = pd.read_csv('input/train.tsv', nrows=N, sep='\t')
# #     if subcats:
# #         cats = df.category_name.str.split('/', expand=True)
# #         df['cat1'] = cats[0]
# #         df['cat2'] = cats[1]
# #         df['cat3'] = cats[2]
# #         df = df.drop(['category_name'], axis=1)
# # #     df.head()

# #     df['brand_id'] = pd.Categorical(df['brand_name']).codes

#     return df

# _models = models.copy()
# _models['ensemble'] = Ensemble()

# #     from collections import Counter
# #     y_pred = [
# # #         list(Counter(items).keys())[0]
# # #         Counter(items).most_common()[0][0]
# #         int(np.mean(items))
# #         for items in 
# #         zip(*predictions)]

# class Router(BaseEstimator):  
#     """An example of classifier"""

#     def __init__(self, name=None, estimator=None, params={}, x_pred=None, y_true=None, **kwargs):
# #         print('INIT', name)
#         self.name = name or 'svc' # type(models[model])
#         self.estimator = estimator
#         self.x_pred = x_pred
#         self.y_true = y_true
#         self.params = params  # or {}
# #         self.model_instance = models[self.name]
#         self.initial_params = _models[self.name].get_params()
# #         print('PARAMS 1', name, self.name, self.model_instance, self.initial_params)
# #         params.update()
#         self.model_instance = type(_models[self.name])(**self.params)
    
# #     def get_model_instance(self):
# #         return type(self.model_instance)(**self.params)
    
#     @property
#     def name(self):
#         return self._name
    
#     @name.setter
#     def name(self, value):
# #         print('SET NAME', value)
#         self._name = value
#         params = _models[self.name].get_params()  # getattr(self, 'initial_params', {}).copy() 
#         params.update(getattr(self, 'params', {}))
# #         print('PARAMS 2', value, getattr(self, 'name', None), getattr(self, 'model_instance', None), params)
#         self.model_instance = type(_models[self._name])(**params)
# #         self.model_instance = type(_models[self._name])(**getattr(self, 'params', {}))
    
#     @property
#     def params(self):
#         return self._params
    
#     @params.setter
#     def params(self, value):
# #         print('SET PARAMS', value)
#         self._params = value
#         params = getattr(self, 'initial_params', {}).copy()
#         params.update(getattr(self, 'params', {}))
# #         print('PARAMS 3', value, getattr(self, 'name', None), getattr(self, 'model_instance', None), params)
#         self.model_instance = type(_models[self._name])(**params)
# #         self.model_instance = type(_models[self._name])(**getattr(self, 'params', {}))
# # #         print('SET PARAMS', value)

    
#     def fit(self, X, y=None):
# #         self.get_model_instance().fit(X, y)
# #         return self
# #         self.model_instance = models[self.name]
#         self.model_instance.fit(X, y)
#         return self

#     def predict(self, X, y=None):
# #         return self.get_model_instance().predict(X)
# #         self.model_instance = models[self.name]
#         return list(map(int, self.model_instance.predict(X)))
       
#     def score(self, X, y=None):
# #         print('SCORE', self.model_instance.predict(X).shape, y.shape)
#         error = mean_squared_error(self.model_instance.predict(X), y)
#         score = 1 / (1 + error)
#         print(self.name, score, error)
#         print('SCORE', self.name, score, self.model_instance.score(X, y))
#         return score
    
#         error = sum([1 if 0.1 < abs(y1 - y2) else 0 for y1, y2 in zip(self.model_instance.predict(X), y)])/y.shape[0]
#         score = 1 / (1 + error)
#         print(self.name, score, error)
#         return score
# #         print('SCORE', X.shape, y.shape)
# # #         return self.model_instance.score(X, y)
# #         if self.estimator:
# #             return self.estimator(self.model_instance)
# # #         print('SCORE', self.model_instance)
# # #         return accuracy_score(y_pred=self.predict(self.x_pred), y_true=self.y_true)
        
# # #         print('SCORE', self.model_instance)
# # # # #         return self.get_model_instance().score(X, y)
# # # #         return self.model_instance.score(X, y)
# # #         self.model_instance = models[self.name]
# #         return(sum(self.model_instance.predict(X)))
# # #         return(sum(self.predict(X))) 
# # #         return accuracy_score(y_pred=self.predict(self.x_pred), y_true=self.y_true)
    
# def completor(df):
#     df = df[:]
#     cats = df.category_name.str.split('/', expand=True)
# #     print('<', df.is_copy())
# #     df.is_copy = False
# #     df.loc[:, 'cat1'] = cats[0]
# #     df.loc[:, 'cat2'] = cats[1]
# #     df.loc[:, 'cat3'] = cats[2]
# #     print('>')
#     df = df.drop(['category_name'], axis=1)
    
#     df = pd.DataFrame(dict(**df.to_dict(), cat1=cats[0], cat2=cats[1], cat3=cats[2]))
    
#     return df

# def _execute_analyze_cycle(model, dataframe, *, x=None, y=None, params=None, features=None, matched=False):
#     ret = dict()
    
# #     def sample_selector(x, count=None, *args, **kwargs):
# # #         print('SELECTOR', x, args, kwargs)
# # #         print('SELECTOR', x, features)
# #         print('SAMPLE', x[:10], count, args, kwargs)
# # #         if count:
# # #             x = x.iloc[:count]
# # #         if count:
# # #             y = y[:count]
# #         return x
        
# #     def selector(X, *args, **kwargs):
#     def feature_selector(x, features=None):
# #         print('SELECTOR', x, args, kwargs)
# #         print('SELECTOR', x, features)
# #         print('FEATURE', type(x), features)
#         if features:
# #             x = x.loc[:, features]
#             x = x.loc[:, features.split(',')]
#         return x
    
#     dataframe = dataframe.dropna()
    
#     x = x or dataframe.drop(['price'], axis=1)
#     y = y or np.ravel(dataframe.loc[:, ['price']])


#     preparation = Pipeline(steps = [
# #             ('x_selector', FunctionTransformer(sample_selector, validate=False, )),
# #             ('y_selector', FunctionTransformer(sample_selector, pass_y=True, validate=False, )),
#         ('feature_selector', FunctionTransformer(feature_selector, validate=False)),

#         ('imputer', SimpleImputer(strategy='most_frequent')),
#         ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
#         ('scaler', StandardScaler()),            
# #         ('model', models[model]) if isinstance(model, str) else model,
# #         ('model', Router('log_reg', estimator=estimator, x_pred=x_test, y_true=y_true))
#     ])
    

    
    
#     x_train, x_test, y_train, y_test = train_test_split(
#         x,
#         y,
#         test_size=0.20, 
#         random_state=42
#     )

#     y_true = np.ravel(y_test)
# #     y_pred = search_cv.predict(x_test)


#     _x_train, _x_test, _y_train, _y_test = train_test_split(
#         preparation.fit_transform(x),
#         y,
#         test_size=0.20, 
#         random_state=42
#     )
#     _y_true = np.ravel(_y_test)


#     def estimator(model):
#         print(model)
# #         x = preparation.fit_transform(x_test)
#         model.fit(_x_train, _y_train)
# #         print(model.predict(x).shape, y_true.shape)
#         return accuracy_score(y_pred=list(map(int, model.predict(_x_test))), y_true=_y_true)

#     pipeline = Pipeline(steps = [
#         ('completor', FunctionTransformer(completor, validate=False)),
# #             ('x_selector', FunctionTransformer(sample_selector, validate=False, )),
# #             ('y_selector', FunctionTransformer(sample_selector, pass_y=True, validate=False, )),
#         ('feature_selector', FunctionTransformer(feature_selector, validate=False)),

#         ('imputer', SimpleImputer(strategy='most_frequent')),
#         ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
#         ('scaler', StandardScaler()),            
# #         ('model', models[model]) if isinstance(model, str) else model,
#         ('model', Router('log_reg', estimator=None, x_pred=x_test, y_true=y_true))
#     ])

#     grid = params or {}

# #     search_cv = RandomizedSearchCV(model, param_grid, cv=5, scoring='accuracy')

#     search_cv = GridSearchCV(pipeline, grid, cv=3, iid=True)             
#     search_cv.fit(x_train, np.array(y_train))

#     ret['search_cv'] = search_cv

#     y_pred = search_cv.predict(x_test)
    
# #     print('PRED', y_pred)
# #     print('TRUE', y_true)

#     ret['accuracy_score'] = 0  # accuracy_score(y_pred=y_pred, y_true=y_true)

#     return ret


# grid = {
# # #     'model__penalty': ['l2', 'none'],
# #     'model__max_iter': [250], # [100, 250, 500, 1000],
    
# #     'x_selector__kw_args': [{'count': count} for count in [250, 500, 750]],
# #     'y_selector__kw_args': [{'count': count} for count in [250, 500, 750]],
#     'feature_selector__kw_args': [
#         {'features': 'brand_name'}, 
#         {'features': 'cat1'}, 
#         {'features': 'cat2'}, 
#         {'features': 'cat3'},
#         {'features': 'shipping'},
#     ],
#     'model__name': list(_models.keys()),
# #     'model__name': ['knei_reg'], # list(models.keys()),  # ['log_reg', 'svc', 'tree'], # [100, 250, 500, 1000],
# # #     'model__params': [{'C': C} for C in [1, 10, 100, 1000, 10000]],
# #     'model__params': [{'n_neighbors': param} for param in range(2, 6)],
# # #     'model__params': [{'max_iter': param} for param in range(100, 1000, 100)],
# }

# # SVC(kernel='linear', C=100)

# # _execute_analyze_cycle(models['log_reg'], df, params=grid)
# # data = _execute_analyze_cycle('log_reg', df[:137], params=grid)
# # data = _execute_analyze_cycle(prepare_data(1000), df[:137], params=grid)
# data = _execute_analyze_cycle('log_reg', prepare_data(300), params=grid)
# # prepare_data(N, subcats=True)

# # data['search_cv'].cv_results_
# # pd.DataFrame(data['search_cv'].cv_results_).loc[:, filter(lambda key: not key.endswith('time'), data['search_cv'].cv_results_.keys())]
# # pd.DataFrame(data['search_cv'].cv_results_).groupby(['param_model__name', 'param_feature_selector__kw_args']).sum()['mean_test_score'].unstack().plot.bar()
# pd.DataFrame(data['search_cv'].cv_results_)
# # {k:(v['features'][0] if k == 'param_feature_selector__kw_args' else v) for k, v in data['search_cv'].cv_results_.items()}
# # {k:print(k, v) for k, v in data['search_cv'].cv_results_.items()}
# # data = {k:[(vv['features'][0] if k == 'param_feature_selector__kw_args' else vv) for vv in v] for k, v in data['search_cv'].cv_results_.items()}

# data = data['search_cv'].cv_results_

# data

# params = list(dict(data)['params'])[0]
# # param_name ='param_model__params'
# param_name = 'param_' + list(params.keys())[0]
# param_name

# data = {k:[(list(vv.values())[0] if k == param_name else vv) for vv in v] for k, v in data.items()}
# data = pd.DataFrame(data)
# data

# # params = data.iloc[0]['params']
# # params.pop('feature_selector__kw_args')
# params.pop('model__name')

# # # # data = pd.DataFrame({k:(v['features'][0] if k == 'param_feature_selector__kw_args' else v) for k, v in data['search_cv'].cv_results_.items()})
# # # data.groupby(['param_model__name', 'param_feature_selector__kw_args']).sum()['mean_test_score'].unstack().plot.bar()
# # param_name = 'param_' + list(params.keys())[0]  # 'param_model__params'
# data.groupby(['param_model__name', param_name]).sum()['mean_test_score'].unstack().plot.bar()

In [ ]:
# by_model_and_feature.groupby(['model', 'feature']).sum()['accuracy_score'].unstack().plot.bar()# (x='feature')

In [ ]:
# # (0.1, Pipeline(memory=None,
# #           steps=[('model',
# #                   XGBClassifier(base_score=0.5, booster='gbtree',
# #                                 colsample_bylevel=1, colsample_bynode=1,
# #                                 colsample_bytree=1, gamma=0, learning_rate=0.1,
# #                                 max_delta_step=0, max_depth=3,
# #                                 min_child_weight=1, missing=None,
# #                                 n_estimators=25, n_jobs=1, nthread=None,
# #                                 objective='multi:softprob', random_state=0,
# #                                 reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
# #                                 seed=None, silent=None, subsample=1,
# #                                 verbosity=1))],
# #           verbose=False))

# df = prepare_data(300)

# for n in range(1, 11, 5):
# # for n in range(1, 206, 5):
#     param_grid = {
#     #     'max_depth': [2, 3, 4],
#     #     'model__n_estimators': range(25, 225, 25), # [25, 50, 100],
#     #     'model__n_estimators': [50], # range(25, 225, 25), # [25, 50, 100],
#         'model__n_estimators': [n], # range(25, 225, 25), # [25, 50, 100],
#     #     'learning_rate': [0.01, 0.025]
#     }
#     # xgb = randomized_cv(xgb.XGBClassifier(), param_grid, x_train, y_train)
#     score = execute_analyze_cycle(
#         xgb.XGBClassifier(),
#     #     df.drop(['price'], axis=1), 
#     #     np.ravel(df.loc[:, ['price']]),
#         df,
#         params=param_grid,
#     )

#     display(n, score['accuracy_score'], score['best_estimator'])

# # score['accuracy_score'], score['best_estimator']

In [ ]:
# # x_train, x_test, y_train, y_test = train_test_split(
# train_test_split(
#     df.drop(['price'], axis=1), 
#     np.ravel(df.loc[:, ['price']]),
#     test_size=0.20, 
#     random_state=42
# )[0]

## Logistic Regression

In [ ]:
# from sklearn.linear_model import LogisticRegression


# # log_reg = LogisticRegression ()
# # log_reg . fit (X , y )
# # y_proba = log_reg . predict_proba ( X_new )

# pipeline = Pipeline(steps = [
# # #     ('imputer', SimpleImputer()),
# #     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
# #     ('scaler', StandardScaler()),
# # #     ('pca', PCA(n_components=2)),
# # #         ('pca', PCA()),
# #     ('log_reg', log_reg),
#     ('log_reg', LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)),
# ])

# # display(X_train.info())
# # display(y_train.info())

# # # pipeline.fit(df.drop(['price'], axis=1), df.loc[:, ['price']])
# # pipeline.fit(x_train, y_train)
# # # y_pred = pipeline.steps[-1][-1].predict(df_test)
# # y_pred = pipeline.predict(x_train)
# # # y_pred = pipeline.predict(df_test)
# # # display(y_pred)

# # X_prepared = pipeline.fit_transform(df)
# # X_prepared.shape, X_prepared
# # X_prepared[0]

# grid = {
#     'log_reg__penalty': ['l2']
# }

# better = GridSearchCV(pipeline, grid, cv=3, iid=True)     
# # display(y_train.shape)
# better.fit(np.array(x_train), np.ravel(np.array(y_train)))  # .reshape(1, y_train.shape[1]), )

# better.best_estimator_

# display(better.best_score_)

# accuracy_score(y_pred = better.predict(x_test), y_true = np.ravel(y_test))
# # accuracy_score(y_pred = better.predict(np.array(test[features])), y_true = test.Survived)

# {'best_score_': 0.09045226130653267, 'accuracy_score': 0.1}
# {'best_score_': 0.10569105691056911, 'accuracy_score': 0.03225806451612903}
# {'best_score_': 0.10569105691056911, 'accuracy_score': 0.03225806451612903}
# {'best_score_': 0.09547738693467336, 'accuracy_score': 0.08}
# {'best_score_': 0.10569105691056911, 'accuracy_score': 0.03225806451612903}

df = prepare_data(1250)
df = df.dropna()

df['brand_id'] = pd.Categorical(df['brand_name']).codes

grid = {
#     'model__penalty': ['l2', 'none'],
    'model__max_iter': [250], # [100, 250, 500, 1000],
#     'x_selector__kw_args': [{'count': count} for count in [250, 500, 750]],
#     'y_selector__kw_args': [{'count': count} for count in [250, 500, 750]],
    'feature_selector__kw_args': [
        {'features': ['brand_name']}, 
#         {'features': ['cat1']}, 
#         {'features': ['cat2']}, 
#         {'features': ['cat3']}
    ],
}

# data = execute_analyze_cycle(LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000), df)
# data = execute_analyze_cycle(LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000), df)
# data = execute_analyze_cycle(LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000), df, 
#   features=['brand_name'])
# data = execute_analyze_cycle(models['log_reg'], df, params=grid, features=['brand_name'])
data = execute_analyze_cycle(models['log_reg'], df, params=grid)


# data = execute_analyze_cycle(
#     LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000), 
#     df.drop(['price'], axis=1), 
#     np.ravel(df.loc[:, ['price']]),
# )

data['best_score'], data['accuracy_score']
# (0.0668103448275862, 0.05128205128205128)
# (0.05818965517241379, 0.05982905982905983)

data['best_estimator']

In [ ]:
# df.iloc[:2]
# pd.DataFrame(data['search_cv'].cv_results_)

In [ ]:
from sklearn.svm import SVC

# # lin_svm = SVC(kernel='linear', C=100).fit(x_train, y_train)

# pipeline = Pipeline(steps = [
# #     ('imputer', SimpleImputer(strategy='most_frequent')),
# #     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
# #     ('scaler',StandardScaler()),
#     ('svc', SVC(kernel='linear', C=100))
# ])
# grid = {
    
# }
# better = GridSearchCV(pipeline, grid, cv=3, iid=True)     
# better.fit(np.array(x_train), np.array(y_train))

# better.best_estimator_

# display(better.best_score_)

# accuracy_score(y_pred = better.predict(x_test), y_true = np.ravel(y_test))

# df = prepare_data(250)

# data = execute_analyze_cycle(
#     SVC(kernel='linear', C=100),
#     df,
# )

# data['best_score'], data['accuracy_score']

scores = {}

for i in [1]:
#     scores[i] = execute_mutly_analyze_cycles(range(250, 1250, 250), SVC(kernel='linear', C=100), trace=True)
    scores[i] = execute_mutly_analyze_cycles(range(250, 500, 250), SVC(kernel='linear', C=100), trace=True)

display(scores[1])
# display(scores[2])

In [ ]:
# df['brand_id'] = pd.Categorical(df['brand_name']).codes

# execute_analyze_cycle(
# #     LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000), 
#     DecisionTreeClassifier(),
#     df.drop(['price'], axis=1).loc[:, ['brand_id']], 
#     np.ravel(df.loc[:, ['price']]),
# )

# # {'best_score_': 0.05025125628140704, 'accuracy_score': 0.04}
# # {'best_score_': 0.05025125628140704, 'accuracy_score': 0.06}

# # pipeline = Pipeline(steps = [
# # #     ('imputer', SimpleImputer(strategy='most_frequent')),
# # #     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
# # #     ('scaler',StandardScaler()),
# #     ('tree', DecisionTreeClassifier())
# # ])
# # grid = {
    
# # }
# # better = GridSearchCV(pipeline, grid, cv=3, iid=True)     
# # better.fit(np.array(x_train), np.array(y_train))

# # better.best_estimator_

# # display(better.best_score_)

# # accuracy_score(y_pred = better.predict(x_test), y_true = np.ravel(y_test))


# {'best_score_': 0.05025125628140704, 'accuracy_score': 0.04}
# {'best_score_': 0.05025125628140704, 'accuracy_score': 0.06}
# 0.06238279784973122


# df = prepare_data(17000)

# # display(df[3700:3750])
# display(df[3780:3783])

scores = pd.DataFrame({'size': [], 'best_score': [], 'accuracy_score': []})
display(scores)
# for i, N in enumerate(range(1000, 10000, 1000)):
# for N in [3000, 3099, 3299, 3499, 3699, 3999, 4200]:
# for N in [3780, 3830, 3960, 3390, 3999, 4410]:
# for N in range(3780, 3810):
for i, N in enumerate([9999]):
#     print(N)
    
    df = prepare_data(N)
    df = df.dropna()

    df['brand_id'] = pd.Categorical(df['brand_name']).codes

    score = execute_analyze_cycle(
        DecisionTreeClassifier(),
        df,
#         df.drop(['price'], axis=1), 
# #                 df.drop(['price'], axis=1).loc[:, ['brand_id']], 
#         np.ravel(df.loc[:, ['price']]),
        matched=True,
        features=['brand_name'],
    )
        
    
#     score = execute_analyze_cycle(
#         DecisionTreeClassifier(),
#         df.drop(['price'], axis=1), 
# #                 df.drop(['price'], axis=1).loc[:, ['brand_id']], 
#         np.ravel(df.loc[:, ['price']]),
#         matched=True,
#     )
    
    scores.loc[i] = (N, score['best_score'], score['accuracy_score'])
    
#     display(score['accuracy_score'])

# scores.plot.plot(x='size', y='score')
scores
# size	best_score	accuracy_score
# 0	9999.00	0.05	0.04

In [ ]:
from operator import itemgetter

# scores.plot(x='size', y='score')

# score
# x_test, y_pred, y_true = itemgetter('x_test', 'y_pred', 'y_true')(score)
# display(df[df['train_id'].isin(x_test[y_pred == y_true]['train_id'])])
# y_pred[y_pred == y_true], y_true[y_pred == y_true], x_test[y_pred == y_true]['train_id']

In [ ]:
df = prepare_data(300)

data = execute_analyze_cycle(
    [
        LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000),
        SVC(kernel='linear', C=100),
        DecisionTreeClassifier()
    ], 
    df
#     LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000), df, features=['brand_name']
)

data['best_score'], data['accuracy_score'], data['accuracy_scores']

In [ ]:
data['predictions'], data['y_pred']

In [ ]:
#

## 3. Провести простые преобразования признаков и добавление простых признаков

### разделить category_name на уровни

### посмотреть на наличие числовых значений в описании и имени

# Задание
1. Сделать baseline submission
  * Исследовать признак price. 
  * Исследовать признак price в зависимости от brand_name или других признаков
2. Реализовать цикл анализа
  * признаки -> модель -> настройка параметров -> лучшая модель и ее значение метрики качества на кросс-валидации
3. Провести простые преобразования признаков и добавление простых признаков
  * разделить category_name на уровни
  * посмотреть на наличие числовых значений в описании и имени
  * ...
4. Составить план по применению нескольких моделей на разных признаках
  * спроектируйте эксперимент. Нужно заранее спланировать порядок перебора признаков и моделей. Потом только писать код. Обратный порядок вызывает необходимость переписывать существующий код, это трата времени
5. Просмотреть 1-5 kernel на kaggle. (только в таком порядке. сначала работаем самостоятельно, потом смотрим идеи других. при появлении опыта можно сразу начинать с них)
6. Скорректировать план
  * убрать пункты, которые кажутся неудачными
  * добавить идеи из kernel, кажущиеся удачными
7. Построить модель, выбрать лучшую
8. Построить ансамбль, настрить парамертры. Сравнить с другими моделями.
9. Применить и засабмитить лучшую на cv модель
10. Прислать блокнот и свой ник в лидерборде